In [6]:
!pip install -r requirements.txt

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=7b58a2c2b7fb7c11c219d1fe9d88d1c6a4acaf1bf9416698ce50993f3147da99
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [1]:
import lxml
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

url1 = "https://www.imdb.com/search/title?count=100&title_type=feature,tv_series&ref_=nv_wl_img_2" # for testing

class IMDB(object):
    
    def __init__(self, url):
        super(IMDB, self).__init__()
        page = get(url)

        self.soup = BeautifulSoup(page.content, 'lxml')

    def articleTitle(self):
        return self.soup.find("h1", class_="header").text.replace("\n","")

    def bodyContent(self):
        content = self.soup.find(id="main")
        return content.find_all("div", class_="lister-item mode-advanced")

    def movieData(self):
        movieFrame = self.bodyContent()
        movieTitle = []
        movieDescription = []
        movieDate = []
        movieRunTime = []
        movieGenre = []
        movieRating = []
        
        for movie in movieFrame:
            movieFirstLine = movie.find("h3", class_="lister-item-header")
            movieTitle.append(movieFirstLine.find("a").text)
            movieDate.append(re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text))
            try:
                movieRunTime.append(movie.find("span", class_="runtime").text[:-4])
            except:
                movieRunTime.append(np.nan)
            try:
                movieGenre.append(movie.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
            except:
                movieGenre.append(np.nan)
            try:
                movieRating.append(movie.find("strong").text)
            except:
                movieRating.append(np.nan)
                
            movieDescription.append(movie.find_all("p", class_="text-muted")[-1].text.lstrip())

            movieNumbers = movie.find_all("span", attrs={"name": "nv"})

        movieData = [movieTitle,movieDescription, movieDate, movieRunTime, movieGenre, movieRating]
        return movieData

In [3]:
def imdb_url(index):
    idx = index + 1
    url = "https://www.imdb.com/search/title/?title_type=feature,tv_series&count=100&start={}&ref_=adv_nxt".format(idx)
    return url

def treat_name(title):
    new_title = ""
    for letter in title:
        if letter == "/":
            new_title = new_title + ""
        else:
            new_title = new_title + letter
    return new_title

In [5]:
for num in range(9400,55000,100):
    url = imdb_url(num)
    id1 = IMDB(url)
    movieData = id1.movieData()
    print(num)

    for i in range(100):
        title = movieData[0][i]
        new_title = treat_name(title)
        descr = movieData[1][i]
        with open("IMDB/{}.txt".format(new_title), 'w') as f:
            print(descr, file=f)

9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
25200
25300
25400
25500
25600
25700
25800
25900
26000
2610

In [13]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

In [15]:
!pwd

/home/jovyan


In [16]:
paths = DisplayablePath.make_tree(Path('IMDB'))
for path in paths:
    print(path.displayable())

IMDB/
├── .DS_Store
├── IMDB.ipynb
├── readme.md
├── requirements.txt
└── src/
    ├── .DS_Store
    └── data/
        ├── 'Allo 'Allo!.txt
        ├── (500) Dias com Ela.txt
        ├── (Dean).txt
        ├── (Des)encanto.txt
        ├── (Des)encontro Perfeito.txt
        ├── +Velozes +Furiosos.txt
        ├── ...E o Vento Levou.txt
        ├── .DS_Store
        ├── 007 - Nunca Mais Outra Vez.txt
        ├── 007 - O Amanhã Nunca Morre.txt
        ├── 007 - O Mundo Não é o Bastante.txt
        ├── 007 - Operação Skyfall.txt
        ├── 007 - Os Diamantes São Eternos.txt
        ├── 007 - Permissão para Matar.txt
        ├── 007 - Quantum of Solace.txt
        ├── 007 - Somente Para Seus Olhos.txt
        ├── 007 - Um Novo Dia Para Morrer.txt
        ├── 007 Contra a Chantagem Atômica.txt
        ├── 007 Contra GoldenEye.txt
        ├── 007 Contra Goldfinger.txt
        ├── 007 Contra o Foguete da Morte.txt
        ├── 007 Contra o Homem com a Pistola de Ouro.txt
        ├── 007 Contra 